In [ ]:
!rm -rf sample_data/

## Install libraries

In [ ]:
!pip install -U --quiet mlfoundry
!pip install -U --quiet servicefoundry

In [ ]:
import os
import getpass

TRUEFOUNDRY_HOST = "https://app.truefoundry.com"
os.environ["TFY_HOST"] = TRUEFOUNDRY_HOST.rstrip('/')
os.environ["TFY_API_KEY"] = getpass.getpass(f"Set API Key (You can generate one at {os.getenv('TFY_HOST')}/settings/) ")
WORKSPACE = input(f"Set Workspace to deploy to (You can get one from {os.getenv('TFY_HOST')}/workspaces) ")

import servicefoundry.core as sfy
sfy.use_server(os.getenv('TFY_HOST'))
sfy.login(api_key=os.getenv('TFY_API_KEY'))

## Load datasets

In [ ]:
# load our datasets as dataframes
import pandas as pd

movie_meta_df = pd.read_csv('https://raw.githubusercontent.com/srihari-tf/recommender-system-tfy/master/movies_metadata.csv')
keywords_df = pd.read_csv('https://raw.githubusercontent.com/srihari-tf/recommender-system-tfy/master/keywords.csv')
ratings_df = pd.read_csv("https://raw.githubusercontent.com/srihari-tf/recommender-system-tfy/master/ratings_small.csv")

movie_meta_df = movie_meta_df[movie_meta_df['id'].isin(ratings_df['movieId'].astype('string'))]

In [ ]:
!curl -O https://raw.githubusercontent.com/srihari-tf/recommender-system-tfy/master/movies_metadata.csv
!curl -O https://raw.githubusercontent.com/srihari-tf/recommender-system-tfy/master/ratings_small.csv

## Clean dataset

In [ ]:
meta_df = movie_meta_df.copy()
meta_df.drop(['belongs_to_collection', 'homepage', 'tagline', 'poster_path', 'overview', 'imdb_id', 'spoken_languages'], inplace=True, axis=1)
meta_df.dropna(inplace=True)

In [ ]:
import ast

column_changes = ['production_companies', 'production_countries', 'genres']
json_shrinker_dict = dict({'production_companies': list(), 'production_countries': list(), 'genres': list()})

for col in column_changes:
    if col == 'production_companies':
        for i in meta_df[col]:
            i = ast.literal_eval(i)
            if len(i) < 1:
                json_shrinker_dict['production_companies'].append(None)

            for element in i:
                json_shrinker_dict['production_companies'].append(element['name'])
                break
    elif col == 'production_countries':
        for i in meta_df[col]:
            i = ast.literal_eval(i)
            if len(i) < 1:
                json_shrinker_dict['production_countries'].append(None)
            for element in i:
                json_shrinker_dict['production_countries'].append(element['iso_3166_1'])
                break
    else:
        for i in meta_df[col]:
            i = ast.literal_eval(i)
            if len(i) < 1:
                json_shrinker_dict['genres'].append(None)

            for element in i:
                json_shrinker_dict['genres'].append(element['name'])
                break

In [ ]:
for i in column_changes:
    meta_df[i] = json_shrinker_dict[i]

meta_df.dropna(inplace=True)
meta_df.head()

## Scale features for clustering

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scalar = MinMaxScaler()
scaled_df = meta_df[['budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count']]
scaled = scalar.fit_transform(meta_df[['budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count']])
scaled_df = pd.DataFrame(scaled, index=scaled_df.index, columns=scaled_df.columns)


## Set up MLFoundry to log params and metrics

In [ ]:
import mlfoundry

mlfoundry.login(api_key=os.getenv('TFY_API_KEY'))
client = mlfoundry.get_client()
MLF_PROJECT_NAME= "movie-clustering-nov-12"

## Find optimal number of clusters and log to MLFoundry

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

clustering_df = scaled_df[['budget', 'popularity', 'revenue', 'runtime', 'vote_average', 'vote_count']]

cluster_sizes = [5, 10, 15, 20, 25]
scores = []
for index, num_cluster in enumerate(cluster_sizes):
    run = client.create_run(project_name=MLF_PROJECT_NAME, run_name=f"num-clusters-{num_cluster}")
    cluster_fit = KMeans(n_clusters=num_cluster, random_state=0).fit(clustering_df)
    score = silhouette_score(clustering_df, cluster_fit.labels_, metric='euclidean', sample_size=None, random_state=None)
    run.log_params({'num_clusters': num_cluster, 'centers': cluster_fit.cluster_centers_})
    run.log_metrics({'silhouette_score': score})
    scores.append(score)
    plt.plot(cluster_sizes[:index+1], scores)
    run.log_plots({'cluster_vs_score': plt})

## Apply KMeans using optimal number of clusters

In [ ]:
def apply_kmeans(df, clusters):
    kmeans = KMeans(n_clusters=clusters, random_state=0)
    cluster_labels = kmeans.fit(df).labels_
    string_labels = ["c{}".format(i) for i in cluster_labels]
    df['cluster_label'] = cluster_labels
    df['cluster_string'] = string_labels

    return df

apply_kmeans(scaled_df, 15)

In [ ]:
movie_desc_df = movie_meta_df.copy()
movie_desc_df = movie_desc_df[['overview', 'original_title', 'id']]
movie_desc_df['overview'] = movie_desc_df['overview'].fillna('')

In [ ]:
from absl import logging

import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model = hub.load(module_url)
print ("module %s loaded" % module_url)
def embed(input):
  return model(input)

In [ ]:
embeddings_df = pd.DataFrame(embed((movie_desc_df['overview'])).numpy())

In [ ]:
embeddings_df.index = movie_desc_df.index

run = client.create_run(project_name=MLF_PROJECT_NAME, run_name=f"embeddings")
cluster_fit = KMeans(n_clusters=15, random_state=0).fit(embeddings_df)
score = silhouette_score(embeddings_df, cluster_fit.labels_, metric='euclidean', sample_size=None, random_state=None)
run.log_metrics({'silhouette_score': score})

## Train a collaborative filtering model using `implicit`

In [ ]:
ratings_df['movieId'] = ratings_df['movieId'].astype("category")
ratings_df['userId'] = ratings_df['userId'].astype("category")

In [ ]:
!pip install implicit

In [ ]:
from scipy.sparse import coo_matrix
import implicit

In [ ]:
r = coo_matrix((ratings_df['rating'], (ratings_df['userId'].cat.codes, ratings_df['movieId'].cat.codes)))

In [ ]:
import implicit

model = implicit.als.AlternatingLeastSquares(factors=25)
model.fit(r)

In [ ]:
user_category_to_code = dict([(category, code) for code, category in enumerate(ratings_df.userId.cat.categories)])

def get_movie_names_from_movie_category_codes(movie_cat_codes):
  ids = [ratings_df['movieId'].cat.categories[i] for i in movie_cat_codes]
  return list(movie_meta_df.loc[movie_meta_df['id'].isin([str(id) for id in ids])].original_title)

def get_recommendation_for_user(user_id):
  user_cat_code = user_category_to_code[user_id]
  return get_movie_names_from_movie_category_codes(model.recommend(user_cat_code, r.tocsr().getrow(user_cat_code))[0])
  

In [ ]:
get_recommendation_for_user(11)

## Save the model and log it to MLFoundry

In [ ]:
model.save('recommendation-model')

In [ ]:
run = client.create_run(project_name=MLF_PROJECT_NAME, run_name=f"cf-model")
run.log_artifact('/content/recommendation-model.npz')
run.log_artifact('/content/ratings_small.csv')
run.log_artifact('/content/movies_metadata.csv')

In [ ]:
# only keep movies in ratings dataset
movie_meta_df = movie_meta_df[movie_meta_df['id'].isin(ratings_df['movieId'].astype('string'))]

ratings_df['movieId'] = ratings_df['movieId'].astype("category")
ratings_df['userId'] = ratings_df['userId'].astype("category")
r = coo_matrix((ratings_df['rating'], (ratings_df['userId'].cat.codes, ratings_df['movieId'].cat.codes)))

user_category_to_code = dict([(category, code) for code, category in enumerate(ratings_df.userId.cat.categories)])
movie_category_to_code = dict([(category, code) for code, category in enumerate(ratings_df.movieId.cat.categories)])

model_local_path = run.download_artifact('recommendation-model.npz')
model = implicit.als.AlternatingLeastSquares(factors=25).load(model_local_path)

def search_movie(name):
  return (movie_meta_df.loc[movie_meta_df['original_title'].str.contains(name, case=False)][['original_title', 'id']]).to_dict('records')

def find_similar_movie(movie_name):
  search_result =search_movie(movie_name)
  if len(search_result) > 0:
    movie_id = search_result[0]['id']
    movie_name = search_result[0]['original_title']
  else:
    return []
  movie_cat_code = movie_category_to_code[int(movie_id)]
  movie_cat_codes = model.similar_items(movie_cat_code)[0]
  ids = [ratings_df['movieId'].cat.categories[i] for i in movie_cat_codes]
  return movie_name, list(movie_meta_df.loc[movie_meta_df['id'].isin([str(id) for id in ids])].original_title)
 
def get_movie_names_from_movie_category_codes(movie_cat_codes):
  ids = [ratings_df['movieId'].cat.categories[i] for i in movie_cat_codes]
  return list(movie_meta_df.loc[movie_meta_df['id'].isin([str(id) for id in ids])].original_title)

def get_recommendation_for_user(user_id):
  user_cat_code = user_category_to_code[int(user_id)]
  movie_cat_codes = model.recommend(user_cat_code, r.tocsr().getrow(user_cat_code))[0]
  ids = [ratings_df['movieId'].cat.categories[i] for i in movie_cat_codes]
  return list(movie_meta_df.loc[movie_meta_df['id'].isin([str(id) for id in ids])].original_title)

In [ ]:
def get_recommendation_for_user_given_movie(user_id, movie_name):
  search_result =search_movie(movie_name)
  if len(search_result) > 0:
    movie_id = search_result[0]['id']
    movie_name = search_result[0]['original_title']
  else:
    return []
  movie_cat_code = movie_category_to_code[int(movie_id)]
  user_cat_code = user_category_to_code[int(user_id)]
  movie_cat_codes = model.rank_items(user_cat_code, r.tocsr().getrow(user_cat_code), model.similar_items(movie_cat_code)[0])[0]
  ids = [ratings_df['movieId'].cat.categories[i] for i in movie_cat_codes]
  return list(movie_meta_df.loc[movie_meta_df['id'].isin([str(id) for id in ids])].original_title)

In [ ]:
get_recommendation_for_user_given_movie(10, 'Shawshank Redemption')

## Deploy the app using ServiceFoundry

In [ ]:
%%writefile predict.py

import os
import mlfoundry
import implicit
import pandas as pd
from scipy.sparse import coo_matrix
from fastapi import FastAPI

app = FastAPI(docs_url="/")

client = mlfoundry.get_client(api_key=os.getenv('TFY_API_KEY'))
run = client.get_run(os.getenv('MLF_RUN_FQN'))

movies_local_path = run.download_artifact('movies_metadata.csv')
ratings_local_path = run.download_artifact('ratings_small.csv')

movie_meta_df = pd.read_csv(movies_local_path)
ratings_df = pd.read_csv(ratings_local_path)

# only keep movies in ratings dataset
movie_meta_df = movie_meta_df[movie_meta_df['id'].isin(ratings_df['movieId'].astype('string'))]

ratings_df['movieId'] = ratings_df['movieId'].astype("category")
ratings_df['userId'] = ratings_df['userId'].astype("category")
r = coo_matrix((ratings_df['rating'], (ratings_df['userId'].cat.codes, ratings_df['movieId'].cat.codes)))

user_category_to_code = dict([(category, code) for code, category in enumerate(ratings_df.userId.cat.categories)])
movie_category_to_code = dict([(category, code) for code, category in enumerate(ratings_df.movieId.cat.categories)])

model_local_path = run.download_artifact('recommendation-model.npz')
model = implicit.als.AlternatingLeastSquares(factors=25).load(model_local_path)

def search_movie(name):
  return (movie_meta_df.loc[movie_meta_df['original_title'].str.contains(name, case=False)][['original_title', 'id']]).to_dict('records')

def find_similar_movie(movie_name):
  search_result =search_movie(movie_name)
  if len(search_result) > 0:
    movie_id = search_result[0]['id']
    movie_name = search_result[0]['original_title']
  else:
    return []
  movie_cat_code = movie_category_to_code[int(movie_id)]
  movie_cat_codes = model.similar_items(movie_cat_code)[0]
  ids = [ratings_df['movieId'].cat.categories[i] for i in movie_cat_codes]
  return movie_name, list(movie_meta_df.loc[movie_meta_df['id'].isin([str(id) for id in ids])].original_title)
 
def get_movie_names_from_movie_category_codes(movie_cat_codes):
  ids = [ratings_df['movieId'].cat.categories[i] for i in movie_cat_codes]
  return list(movie_meta_df.loc[movie_meta_df['id'].isin([str(id) for id in ids])].original_title)

@app.get("/recommendations")
def get_recommendation_for_user(user_id):
  user_cat_code = user_category_to_code[int(user_id)]
  movie_cat_codes = model.recommend(user_cat_code, r.tocsr().getrow(user_cat_code))[0]
  ids = [ratings_df['movieId'].cat.categories[i] for i in movie_cat_codes]
  return list(movie_meta_df.loc[movie_meta_df['id'].isin([str(id) for id in ids])].original_title)

@app.get("/similar-movies")
def get_similar_movies_for_user(user_id, movie_name):
  search_result =search_movie(movie_name)
  if len(search_result) > 0:
    movie_id = search_result[0]['id']
    movie_name = search_result[0]['original_title']
  else:
    return []
  movie_cat_code = movie_category_to_code[int(movie_id)]
  user_cat_code = user_category_to_code[int(user_id)]
  movie_cat_codes = model.rank_items(user_cat_code, r.tocsr().getrow(user_cat_code), model.similar_items(movie_cat_code)[0])[0]
  ids = [ratings_df['movieId'].cat.categories[i] for i in movie_cat_codes]
  return list(movie_meta_df.loc[movie_meta_df['id'].isin([str(id) for id in ids])].original_title)

In [ ]:
%%writefile requirements.txt

mlfoundry
implicit
pandas
scikit-learn
fastapi
uvicorn

In [ ]:
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(name)-12s %(levelname)-8s %(message)s', force=True)

from servicefoundry import Build, Service, PythonBuild


service = Service(
    name="movie-pred-svc",
    image=Build(
        build_spec=PythonBuild(
            command="uvicorn predict:app --port 8000 --host 0.0.0.0",
            requirements_path="requirements.txt",
        )
    ),
    ports=[{"port": 8000}],
    env={
        "TFY_HOST": os.getenv("TFY_HOST"),
        "TFY_API_KEY": os.getenv("TFY_API_KEY"),
        # replace with your RUN FQN
        "MLF_RUN_FQN": 'truefoundry/akash/movie-clustering-nov-12/cf-model'
    },
)
deployment = service.deploy(workspace_fqn=WORKSPACE)